In [1]:
import sys
sys.path.append('../../')

In [2]:
from preprocessing.frontalizer import Frontalizer

In [29]:
import pandas as pd
males = pd.read_csv('/home/jos/datasets/facescrub/facescrub_actors.txt', delimiter='\t')
females = pd.read_csv('/home/jos/datasets/facescrub/facescrub_actresses.txt', delimiter='\t')

men = set(males['name'])
women = set(females['name'])
all_people = ['_'.join(p.split()) for p in set(list(men) + list(women))]

In [30]:
print(sorted(all_people))

['Aaron_Eckhart', 'Adam_Brody', 'Adam_McKay', 'Adam_Sandler', 'Adrianne_León', 'Adrien_Brody', 'Adrienne_Barbeau', 'Adrienne_Frantz', 'Aisha_Hinds', 'Al_Pacino', 'Alan_Alda', 'Alan_Arkin', 'Alan_Rickman', 'Alec_Baldwin', 'Alexander_Skarsgård', 'Alfred_Molina', 'Alice_Krige', 'Alley_Mills', 'Allison_Janney', 'Alyson_Hannigan', 'Alyssa_Milano', 'Amanda_Bearse', 'Amaury_Nolasco', 'America_Ferrera', 'Amy_Davidson', 'Andrea_Anders', 'Andrea_Bogart', 'Andrea_Bowen', 'Andy_Garcia', 'Andy_Richter', 'Andy_Serkis', 'Angell_Conwell', 'Angie_Harmon', 'Anne_Hathaway', 'Annie_Ilonzeh', 'Anthony_Hopkins', 'Anthony_Stewart_Head', 'Antonio_Banderas', 'Arnold_Schwarzenegger', 'Arnold_Vosloo', 'Ashley_Benson', 'Ashley_Johnson', 'Ashley_Jones', 'Ashton_Kutcher', 'Audra_McDonald', 'Audrey_Landers', 'Barbara_Carrera', 'Ben_Affleck', 'Ben_Kingsley', 'Ben_McKenzie', 'Ben_Stiller', 'Benicio_Del_Toro', 'Bernard_Hill', 'Bernie_Mac', 'Bill_Cosby', 'Bill_Hader', 'Bill_Murray', 'Bill_Nighy', 'Billy_Bob_Thornton', '

In [33]:
%matplotlib inline
import os
import os.path as path
from scipy.misc import imread, imresize
import matplotlib.pylab as plt
import dlib
import numpy as np
import h5py
detector = dlib.get_frontal_face_detector()

frontalizer = Frontalizer()
base = '/home/jos/datasets/facescrub/download'
people = os.listdir(base)

def crop_safe(img, d, factor=0.4):
    width = d.right() - d.left()
    height = d.bottom() - d.top()
    left = int(d.left() - width * factor)
    right = int(d.right() + width * factor)
    bottom = int(d.bottom() + height * factor)
    top = int(d.top() - height * factor)
    left = max(0, left)
    right = min(img.shape[1], right)
    top = max(0, top)
    bottom = min(img.shape[0], bottom)
    return img[top:bottom, left:right]

n_total_images = 0

for person in people:
    image_files = [fnm for fnm in os.listdir(path.join(base, person)) if '.jpg' in fnm]
    n_total_images += len(image_files)

out_shape = (n_total_images, 227, 227, 3)

with h5py.File('/home/jos/datasets/facescrub/facescrub.h5', 'w') as h5f:
    images = h5f.create_dataset('images', shape=out_shape, dtype=np.float32)
    labels_name = h5f.create_dataset('labels/name', shape=[n_total_images, len(all_people)], dtype=np.float32)
    labels_gender = h5f.create_dataset('labels/gender', shape=[n_total_images, 2], dtype=np.float32)

    def one_hot(i, n):
        return np.asarray([0 if j != i else 1 for j in range(n)])

    index = 0
    for person in people:
        image_files = [fnm for fnm in os.listdir(path.join(base, person)) if '.jpg' in fnm]
        label_n = one_hot(all_people.index(person), len(all_people))
        label_g = one_hot(person in men, 2)
        for fnm in image_files:
            img = imread(path.join(base, person, fnm))
            _, frontal = frontalizer.transform(img)
            dets = detector(frontal, 1)
            if len(dets) != 1:
                print("failure")
                continue
            cropped = crop_safe(frontal, dets[0])
            images[index] = imresize(cropped, (227, 227))
            labels_name[index] = label_n
            labels_gender[index] = label_g
            index += 1
print(n_total_images, index)
        


/home/jos/anaconda3/envs/lws/lib/python3.6/site-packages/ipykernel/__main__.py:50: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Number of faces detected: 2
query image shape: (250, 280, 3)


/home/jos/anaconda3/envs/lws/lib/python3.6/site-packages/ipykernel/__main__.py:54: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Number of faces detected: 1
query image shape: (488, 391, 3)
Number of faces detected: 1
query image shape: (1392, 1200, 3)


IndexError: Index out of range

In [22]:
h5f.close()

In [ ]:


dets = detector(img, 1)

    print("Number of faces detected: {}".format(len(dets)))
    for i, d in enumerate(dets):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(